In [1]:
from generate_declarative_sentences import generate_declarative_statements
from categorise_sentences import QuestionCategoryAssigner, StatementCategoryAssigner
from dataset_processing import DatasetPreProcessor

model_name = "gemma3:1b-it-qat"
#generate a small sample of declarative sentences from the NYC SQuAD dataset
declarative_sentences_dataset = generate_declarative_statements(5, model_name)
#use this dataset to assign categories to the questions and statements
question_assigner = QuestionCategoryAssigner(declarative_sentences_dataset)
question_assigner.generate_statement_categories(model_name)
statement_assigner = StatementCategoryAssigner(declarative_sentences_dataset)
statement_assigner.generate_statement_categories(model_name)
declarative_sentences_dataset

2025-11-09 15:14:33,380 - root - INFO - Logging initialized. Log file: /home/chris/logs/cognitive_language_model_logs/run_20251109_151433.log
2025-11-09 15:14:33,403 - generate_declarative_sentences - INFO - 🚀 Starting project: Training-and-evaluating-cognitive-language-models
2025-11-09 15:14:33,403 - generate_declarative_sentences - INFO - 
--- Model Config ---
2025-11-09 15:14:33,404 - generate_declarative_sentences - INFO -    options: {'num_ctx': 4096, 'repeat_last_n': 64, 'repeat_penalty': 1.5, 'temperature': 0.0, 'seed': 42, 'num_predict': 100, 'top_k': 1, 'top_p': 0.1, 'min_p': 0.0}
2025-11-09 15:14:33,404 - generate_declarative_sentences - INFO -    model: gemma3:1b-it-qat
2025-11-09 15:14:33,404 - generate_declarative_sentences - INFO - 
--- Final Settings (as JSON) ---
2025-11-09 15:14:33,404 - generate_declarative_sentences - INFO - {
  "WANDB_API_KEY": "super_secret_key_12345abcdef",
  "project_name": "Training-and-evaluating-cognitive-language-models",
  "dataset": {
    

,id,title,context,question,answers,declarative_statement,question_category,statement_category
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ...",The Virgin Mary allegedly appeared to Saint Be...,Subject-Verb-Object,Subject-Verb-Object
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe...",a copper statue of Christ is in front of the N...,Subject-Verb-Object,Subject-Verb-Object
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'...",The Basilica of the Sacred Heart at Notre Dame...,Subject-Verb-Object,Subject-Verb-Object
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...,the Grotto at Notre Dame is a Marian place of ...,Subject-Verb-Object,Subject-Verb-Object
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...,a golden statue of the Virgin Mary sits on top...,Subject-Verb-Object,Subject-Verb-Object


In [2]:
#apply the formatting rules to the dataset
datasetPreProcessor = DatasetPreProcessor(declarative_sentences_dataset)
datasetPreProcessor.preprocess_data()
declarative_sentences_dataset

,id,title,context,question,declarative_statement,question_category,statement_category,answer,declarative_statement_formatted,question_formatted,answer_formatted
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin-Mary allegedly appear i...,The-Virgin-Mary allegedly appeared to Saint-Be...,Subject-Verb-Object,Subject-Verb-Object,Saint-Bernadette-Soubirous,the Virgin Mary allegedly appeared to Saint Be...,? to whom did the Virgin Mary allegedly appear...,Saint Bernadette Soubirous
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre-Dame-Main-Building?,a copper statue of Christ is in front of the N...,Subject-Verb-Object,Subject-Verb-Object,a copper statue of Christ,a copper statue of Christ is in front of the N...,? what is in front of the Notre Dame Main Buil...,a copper statue of Christ
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The-Basilica of the Sacred heart at Notre-Dame...,The-Basilica of the Sacred-Heart at Notre-Dame...,Subject-Verb-Object,Subject-Verb-Object,the Main-Building,the Basilica of the Sacred Heart at Notre Dame...,? the Basilica of the Sacred heart at Notre Da...,the Main Building
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre-Dame?,the Grotto at Notre-Dame is a Marian place of ...,Subject-Verb-Object,Subject-Verb-Object,a Marian place of prayer and reflection,the Grotto at Notre Dame is a Marian place of ...,? what is the Grotto at Notre Dame,a Marian place of prayer and reflection
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main-Building at Notre...,a golden statue of the Virgin-Mary sits on top...,Subject-Verb-Object,Subject-Verb-Object,a golden statue of the Virgin-Mary,a golden statue of the Virgin Mary sits on top...,? what sits on top of the Main Building at Not...,a golden statue of the Virgin Mary


In [ ]:
#todo add lemmatisation and POS tagging to columns
# use the POS tagging to identify goals as part of command generation